In [3]:
import numpy as np
import pandas as pd
import sklearn.ensemble
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
plt.style.use('ggplot')
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [5]:
from Feat_eng import Y_train, X_train, Y_test, X_test, c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test  

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [6]:
gbm = GradientBoostingRegressor()
gbm.set_params(learning_rate = 0.01, n_estimators = 500, subsample = 0.7)

gbm.fit(X_train,Y_train)

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingRegressor(learning_rate=0.01, n_estimators=500, subsample=0.7)

In [7]:
# print(gbm.best_score_)
print(gbm.score(X_train,Y_train))
print(gbm.score(X_test,Y_test))

0.9906981472517199
0.9888473164072482


In [8]:
pred = gbm.predict(X_train)
pred_unlog = np.exp(pred)

pred_test = gbm.predict(X_test)
pred_test_unlog = np.exp(pred_test)

Y_unlog = np.exp(Y_train)
Y_test_unlog = np.exp(Y_test)



print("Train RMSE = ", np.sqrt(mean_squared_error(pred_unlog, Y_unlog)))
train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
print(train_RMSE)
print("Test RMSE = ", np.sqrt(mean_squared_error(pred_test_unlog, Y_test_unlog)))

Train RMSE =  67.79061773789292
67.79061773789292
Test RMSE =  78.71095569100504


In [10]:
# GradientBoostingRegressor(learning_rate=0.01, n_estimators=1000, subsample=0.5)

# grid_para_tree = [{'learning_rate': [0.01,0.02,0.03,0.04],
#                   'subsample'    : [0.9, 0.5, 0.2, 0.1],
#                   'n_estimators' : [50,100,250,500,1000]
# #                   'max_depth'    : [4,6,8,10]

                   
grid_para_tree = [{'learning_rate': [0.005, 0.01, 0.015, 0.02],
                  'subsample'    : [0.6, 0.5, 0.4, 0.2],
                  'n_estimators' : [500, 1000,1500],
#                  'max_depth'    : [2,5,10]
                 }]

boost = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 2, 
                                n_jobs = -1, n_iter = 10)

boost.fit(X_train, Y_train)

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=2,
                   estimator=GradientBoostingRegressor(learning_rate=0.01,
                                                       n_estimators=500,
                                                       subsample=0.7),
                   n_jobs=-1,
                   param_distributions=[{'learning_rate': [0.005, 0.01, 0.015,
                                                           0.02],
                                         'n_estimators': [500, 1000, 1500],
                                         'subsample': [0.6, 0.5, 0.4, 0.2]}])

In [11]:
boost.best_estimator_

GradientBoostingRegressor(learning_rate=0.015, n_estimators=500, subsample=0.4)

In [12]:
print(boost.best_score_)
print(boost.score(X_test,Y_test))

0.984497830672266
0.9892350005166933


In [13]:
pred = boost.predict(X_train)
pred_unlog = np.exp(pred)

pred_test = boost.predict(X_test)
pred_test_unlog = np.exp(pred_test)

Y_unlog = np.exp(Y_train)
Y_test_unlog = np.exp(Y_test)



train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
print(train_RMSE)
print("Test RMSE = ", np.sqrt(mean_squared_error(pred_test_unlog, Y_test_unlog)))

62.49830445640758
Test RMSE =  77.06691667768942


## Same for all Clusters

In [14]:
grid_para_tree = [{'learning_rate': [0.005, 0.01, 0.015, 0.02],
                  'subsample'    : [0.6, 0.5, 0.4, 0.2],
                  'n_estimators' : [500,1000, 1500]
#                   'max_depth'    : [4,6,8,10]
                 }]



In [15]:
boost1 = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 2, 
                                n_jobs = -1, n_iter = 20)

boost1.fit(c1_train, Y_c1train)

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


KeyboardInterrupt: 

In [ ]:
boost2 = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 2, 
                                n_jobs = -1, n_iter = 20)

boost2.fit(c2_train, Y_c2train)

In [ ]:
boost3 = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 2, 
                                n_jobs = -1, n_iter = 20)

boost3.fit(c3_train, Y_c3train)

## XGBoost

In [16]:
from xgboost import XGBRegressor

model = XGBRegressor(random_state = 0)

model.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
pred = model.predict(X_train)
pred_unlog = np.exp(pred)

pred_test = model.predict(X_test)
pred_test_unlog = np.exp(pred_test)

Y_unlog = np.exp(Y_train)
Y_test_unlog = np.exp(Y_test)

In [18]:
print("Train RMSE = ", np.sqrt(mean_squared_error(pred_unlog, Y_unlog)))
train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
train_RMSE

print("Test RMSE = ", np.sqrt(mean_squared_error(pred_test_unlog, Y_test_unlog)))


Train RMSE =  17.973555451043516
Test RMSE =  79.4661503888568


In [19]:
print(model.score(X_train, Y_train))
print(model.score(X_test, Y_test))

0.9992656430974624
0.987873475167241


In [38]:
# {'subsample': 0.75,
#  'n_estimators': 200,
#  'min_child_weight': 6,
#  'max_leaves': 6,
#  'max_depth': 7,
#  'learning_rate': 0.1,
#  'gamma': 0.8,
#  'colsample_bytree': 0.95,
#  'colsample_bynode': 0.8,
#  'colsample_bylevel': 0.8}



hyperparameters = {
    'colsample_bylevel': [0.75, 0.8, 0.85],
    'colsample_bynode': [0.7, 0.8, 0.85],
    'colsample_bytree': [0.75, 0.8, 0.85], 
    'learning_rate': [0.025, 0.05, 0.1, 0.15],
    'max_depth': [5, 6, 7],
    'max_leaves': [5, 6, 7],
    'min_child_weight': [5, 6, 7],
    'subsample': [0.7, 0.75, 0.8],
    'gamma':[0.75, 0.8, 0.85],
    'n_estimators': [150, 200, 250]
}

gs_boost = XGBRegressor(random_state=0, reg_lambda = 0)
model = RandomizedSearchCV(gs_boost, hyperparameters, cv = 2, n_jobs=-1, n_iter = 10)

# model.fit(X_train, Y_train, eval_set = [(X_ver,Y_ver)], early_stopping_rounds = 20)
model.fit(X_train, Y_train) #, early_stopping_rounds = 20)


RandomizedSearchCV(cv=2,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n...
                                          validate_parameters=None,
                                          verbosity=None),
                   n_jobs=-1,
                   param_distribu

In [39]:
pred = model.predict(X_train)
pred_unlog = np.exp(pred)

pred_test = model.predict(X_test)
pred_test_unlog = np.exp(pred_test)

Y_unlog = np.exp(Y_train)
Y_test_unlog = np.exp(Y_test)

In [40]:
print("Train RMSE = ", np.sqrt(mean_squared_error(pred_unlog, Y_unlog)))
train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
train_RMSE

print("Test RMSE = ", np.sqrt(mean_squared_error(pred_test_unlog, Y_test_unlog)))


Train RMSE =  101.41441914302706
Test RMSE =  112.51784103224558


In [41]:
print(model.score(X_train, Y_train))
print(model.score(X_test, Y_test))

0.9846471630489323
0.9825107126920777


In [42]:
model.best_estimator_
model.best_params_

{'subsample': 0.8,
 'n_estimators': 250,
 'min_child_weight': 6,
 'max_leaves': 6,
 'max_depth': 7,
 'learning_rate': 0.05,
 'gamma': 0.85,
 'colsample_bytree': 0.85,
 'colsample_bynode': 0.85,
 'colsample_bylevel': 0.85}

In [ ]:

# model = XGBRegressor(random_state=0, subsample = 0.85, n_estimators = 500,

#                     learning_rate = 0.2, colsample_bytree = 0.7,

#                     colsample_bylevel = 0.75, max_depth = 4,

#                     min_child_weight = 5, max_delta_step = 0,

#                     max_leaves = 10, gamma=1)



# print(model)

# model.fit(X_train, Y_train, eval_set = [(X_test,Y_test)], early_stopping_rounds = 20)

# predictions = model.predict(X_train)

# predictions_test = model.predict(X_test)

# print(np.sqrt(mean_squared_error(predictions, Y_train)))

# print(np.sqrt(mean_squared_error(predictions_test, Y_test)))

# print(model.score(X_train, Y_train))

# print(model.score(X_test, Y_test))

# # hyperparameters = {

# #     'max_depth':[3,4,5,6,7,8,9,10],

# #     'colsample_bylevel': [0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95],

# #     'colsample_bynode': [0, 0.2, 0.4, 0.6, 0.8, 1],

# #     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 

# #     'learning_rate': [0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],

# #     'max_depth': [0, 1, 2, 3, 4, 5, 6, 7],

# #     'max_leaves': [2, 4, 6, 8, 10, 15, 20],

# #     'min_child_weight': [1,2,3,4,5],

# #     'subsample': [0.2, 0.4, 0.5, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9],

# #     'gamma':[0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],

# #     'n_estimators': [5, 10, 50, 100, 200, 350, 500]

# # }

# ​

# hyperparameters = {

#     'colsample_bylevel': [0.45, 0.55, 0.65, 0.75, 0.85],

#     'colsample_bynode': [ 0.2, 0.4, 0.6, 0.8, 1],

#     'colsample_bytree': [0.5, 0.6, 0.7, 0.8], 

#     'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.25],

#     'max_depth': [4, 6, 8, 10],

#     'max_leaves': [4, 6, 8, 10, 12],

#     'min_child_weight': [4, 5, 6],

#     'subsample': [0.7, 0.75, 0.8, 0.85, 0.9],

#     'gamma':[0.4, 0.6, 0.8, 1],

#     'n_estimators': [300, 400, 500, 600]

# }

# ​

# # hyperparameters = {

# #     'colsample_bynode': [0.1, 0.2, 0.4],

# #     'max_depth': [4, 6],

# #     'max_leaves': [4, 6, 8, 10, 12],

# #     'min_child_weight': [4, 5, 6],

# #     'gamma':[0.8, 1]

# # }

# gs_boost = XGBRegressor(random_state=0, reg_lambda = 0)

# # model = XGBRegressor(random_state=0, subsample = 0.85, n_estimators = 500,

# #                     learning_rate = 0.2, colsample_bytree = 0.7,

# #                     colsample_bylevel = 0.75, max_depth = 4,

# #                     min_child_weight = 5, max_delta_step = 0,

# #                     max_leaves = 10, gamma=1)

# ​

# # 3.807204614385419

# # 6.919303509257749

# # 0.9366801340541325

# # 0.7513589994475436

# ​

# # XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.45,

# #              colsample_bynode=0.6, colsample_bytree=0.8, gamma=0.6, gpu_id=-1,

# #              importance_type='gain', interaction_constraints='',

# #              learning_rate=0.05, max_delta_step=0, max_depth=4, max_leaves=8,

# #              min_child_weight=5, missing=nan, monotone_constraints='()',

# #              n_estimators=600, n_jobs=4, num_parallel_tree=1, random_state=0,

# #              reg_alpha=0, reg_lambda=0, scale_pos_weight=1, subsample=0.7,

# #              tree_method='exact', validate_parameters=1, verbosity=None)

# model = RandomizedSearchCV(gs_boost, hyperparameters, cv = 5, n_jobs=-1, n_iter = 50)

# ​

# model.fit(X_train, Y_train, eval_set = [(X_ver,Y_ver)], early_stopping_rounds = 20)


# predictions = model.predict(X_train)

# predictions_test = model.predict(X_ver)

# print(np.sqrt(mean_squared_error(predictions, Y_train)))

# print(np.sqrt(mean_squared_error(predictions_test, Y_ver)))

# print(model.best_score_)

# print(model.score(X_ver, Y_ver))

# ​

# # model.best_score_

# 3.9975022688466892
# 7.30568867372593
# 0.727762928688851
# 0.7536665207233895

# print(model.score(X1_test, Y_test))

# predictions_test = model.predict(X1_test)

# print(np.sqrt(mean_squared_error(predictions_test, Y_test)))

# ​

# 0.7156295431127093
# 7.646924351377425

